## Imports

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Preprocesing

In [ ]:
csv_path = '../data/household_data_60min_singleindex.csv'

df = pd.read_csv(csv_path, parse_dates=["utc_timestamp", "cet_cest_timestamp"], index_col="utc_timestamp")

#residential building in suburban area
columns_to_drop = [col for col in df.columns if not col.startswith('DE_KN_residential2')]

# Clear data
df = df.drop(columns= columns_to_drop)
df = df.dropna(how='all')

#Data shape and sample
print(f'Data shape: {df.shape}')
df.head()

## Processing data

In [ ]:
df = df.diff().fillna(0)
#sum days 
df = df.resample('D').sum()
df.head()

In [ ]:
daily_usage_filtered = []
K = 8 #standard deviations

for column_name in df:
    mean = df[column_name].mean()
    std_dev = df[column_name].std()

    # Define a range for normal values (e.g., within 2 standard deviations)
    lower_bound = mean - K * std_dev
    upper_bound = mean + K * std_dev

    # Filter out values outside the normal range
    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [ ]:
df.head()

In [ ]:
grid_import = df['DE_KN_residential2_grid_import']

df = df.drop(columns= 'DE_KN_residential2_grid_import')

In [ ]:
df['sum'] = df.sum(axis=1, numeric_only=True)
df.head()

## Graphs

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(df.index, df['DE_KN_residential2_circulation_pump'], label='Pompa obiegowa')
plt.plot(df.index, df['DE_KN_residential2_freezer'], label='Zamrażarka')

plt.xlabel('Data')
plt.ylabel('Zużycie')
plt.title('Zużycie energi elektrycznej w czasie')

plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(df.index, df['sum'], label='Domostwo')

plt.xlabel('Data')
plt.ylabel('Zużycie energii [kWh]')
plt.title('Zużycie energi elektrycznej w czasie')

plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Normality

In [ ]:
from scipy.stats  import normaltest

statistic, p_value = normaltest(df['sum'])

print(f'Test statistic: {statistic}')
print(f'P-values, if lower than 0.05 then its normal distribution: {p_value}')

## Split data 

In [ ]:
# Prepare features and target
features = (df.index.astype(np.int64) // 10**9).values.reshape(-1,1)
#features = grid_import.values.reshape(-1,1)
target = df['sum']

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=11)


## LinearRegression model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create Linear Regression model
model_LR = LinearRegression()

# Train the model on the training data
model_LR.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model_LR.predict(X_test)

# Evaluate model performance using Mean Squared Error
mse_LR = mean_squared_error(y_test, y_pred)

In [ ]:
plt.scatter(y_test, y_pred, label='Predicted vs Actual')
plt.plot(y_test, y_test, color='r', label='Perfect Prediction Line')
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted Values")
plt.legend()
plt.show()

In [ ]:
residuals = y_test - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Residual Plot")
plt.show()

In [ ]:
coefficients = model_LR.coef_
intercept = model_LR.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

## DecisionTreeRegressor Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model_tree = DecisionTreeRegressor(max_depth=8)  # You can adjust hyperparameters like max_depth
model_tree.fit(X_train, y_train)

In [ ]:
y_pred = model_tree.predict(X_test)
mse_DTR = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse_DTR)

In [ ]:
plt.scatter(y_test, y_pred, label='Predicted vs Actual')
plt.plot(y_test, y_test, color='r', label='Perfect Prediction Line')

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted Values")
plt.show()

In [ ]:
residuals = y_test - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
plt.hist(residuals, bins=10)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()

In [ ]:
X = df.index.hour.values.reshape(-1, 1)
xticks = pd.date_range(start=df.index.min(), end=df.index.max(),freq='D')

In [ ]:
def eval_on_features(features, target, regressor, n_train):
    X_train, X_test = features[:n_train], features[n_train:]

    y_train, y_test = target[:n_train], target[n_train:]
    regressor.fit(X_train, y_train)

    print('Zestaw testowy R^2: {:.2f}'.format(regressor.score(X_test, y_test)))
    
    y_pred = regressor.predict(X_test)
    y_pred_train = regressor.predict(X_train)

    mse = mean_squared_error(y_test, y_pred)

    plt.figure(figsize=(10,3))
    plt.xticks(range(0, len(X), 20), rotation=45, ha='left')
    plt.plot(range(n_train), y_train, label='dane uczące')
    plt.plot(range(n_train, (len(y_test) + n_train)), y_test, '-', label='dane testowe')
    plt.plot(range(n_train), y_pred_train, '--', label='prognoza dla uczących')
    plt.plot(range(n_train, len(y_test) + n_train), y_pred, '--', label='prognoza dla testowych')
    plt.legend()
    plt.xlabel('Data')
    plt.ylabel('Wykorzystanie energii [kWh]')
    print("Mean Squared Error:", mse)

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, random_state=22)

n_train = 100
df = df[50:1050]
y = df['sum']
y = y.values
X_hour_week = np.hstack([df.index.dayofweek.values.reshape(-1,1), df.index.hour.values.reshape(-1,1)])


In [ ]:
eval_on_features(X_hour_week, y, regressor, n_train)

## Ridge

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge

enc = OneHotEncoder()
X_hour_week_onehot = enc.fit_transform(X_hour_week).toarray()

In [ ]:
eval_on_features(X_hour_week_onehot, y, Ridge(), n_train)

## Linear Regression

In [ ]:
eval_on_features(X_hour_week, y, LinearRegression(),n_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model_tree = DecisionTreeRegressor(max_depth=8)  # You can adjust hyperparameters like max_depth

eval_on_features(X_hour_week, y, model_tree,n_train)

## Conclusion

In [ ]:
print("Mean Squared Error for Linear Regresion:", mse_LR)
print("Mean Squared Error for Decision Tree Regression:", mse_DTR)
#print("Mean Squared Error (Gaussian Process):", mse_GPR)
print("Mean Squared Error for Random Forest Regression:", mse_RFR)